# Natural Language Processing with ConvNets

🎯 The goal of this challenge is to show you that we can also use 1D Convolutional Filters to scan sentences 🧨 (_instead of RNN_).

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [1]:
!pip freeze | grep gensim

gensim==4.2.0


# The data

❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [2]:
######################################
### Run this cell to load the data ###
######################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2023-11-23 20:03:49.953484: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Remember that to do NLP, you need to go through one of the following options, as shown here: 

<img src="embedding_or_RNN.png" width="700px" />

But in both cases, you can replace the recurrent layer (top part) by a CNN layer. 

👉 We will try both options, starting from the one on the left were the embeddings are learned within the network.

# Part 1 : Concatenate a Keras Embedding with a Conv1D🔥 

Let's train a fancy network here. 

Each of your words is represented by a vector of size N (the size of your embedding). Therefore, as a sentence is a sequence of words, it is represented by a matrix (number of words, N). Consequently, all your sentences are actually represented as matrices once embedded.

If you think about it, an image is also a matrix. Said differently, you may represent your sentence of word as a matrix, where each column (or row, depending on how you want to look at it) is a word, and each row (or each column) corresponds to a coordinate in the embedding space. As shown here

<img src="image_comparison.png" width="500px" />

Well, in that case, as these are close to images, why not using convolutions on them? Yes, convolutions!
But, be careful. In the case of images, convolutions are 2-dimensional as the filters scan these pictures left to right and top to bottom. In the case of our sentences, we want the kernel to move _only_ in word by word, hence left to right. It wouldn't make sense to move the kernels top to bottom as _one vector = one word_.

So let's create a model that uses convolutions.

## First, the data

❓ **Question** ❓ You will need to prepare your data. First, tokenize them. Then, you need to pad them (use a value `maxlen` equal to 150). You also might need to compute the size of your vocabulary ;)

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [6]:
# YOUR CODE HERE
tk = Tokenizer()
tk.fit_on_texts(X_train)
X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)

In [8]:
X_train_pad = pad_sequences(X_train_token,maxlen=150)
X_test_pad = pad_sequences(X_test_token,maxlen=150)

## Using 1D Convolution.

❓ **Question** ❓ Define a model that has :
- an `Embedding` layer: 
    - `input_dim` = `vocab_size + 1`
    - `output_dim` is the embedding space dimension
    - `mask_zero` has to be set to `True`. 
    - Here, for computational reasons, set `input_length` to the maximum length of your observations (that you just defined in the previous question).
- a `Conv1D` layer 
- a `Flatten` layer
- a `Dense` layer
- an output layer

Compile the model accordingly.

❗ **Remark** ❗ The size of the `Conv1D` kernel corresponds exactly to the number of side-by-side words (tokens) each kernel will take into account ;)

In [12]:
X_train_pad.shape

(2500, 150)

In [9]:
vocab_size = len(tk.word_index)

In [29]:
# YOUR CODE HERE
model_cnn = Sequential()

model_cnn.add(layers.Embedding(input_dim=vocab_size+1,
                           input_length=150,
                           output_dim=50,
                           mask_zero=True))
model_cnn.add(layers.Conv1D(30,kernel_size=4,activation='tanh'))
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(5,activation='relu'))
model_cnn.add(layers.Dense(1,activation='sigmoid'))

model_cnn.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

❓ **Question** ❓ Look at the number of parameters. You can compare it to the model that you had in previous exercises (esp. the first one)

In [30]:
# YOUR CODE HERE
model_cnn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 150, 50)           1521000   
                                                                 
 conv1d_4 (Conv1D)           (None, 147, 30)           6030      
                                                                 
 flatten_4 (Flatten)         (None, 4410)              0         
                                                                 
 dense_4 (Dense)             (None, 5)                 22055     
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                                 
Total params: 1,549,091
Trainable params: 1,549,091
Non-trainable params: 0
_________________________________________________________________


❓ **Question** ❓ Fit your model with a stopping criterion, and evaluate it on the test data.

You will probably notice that it is ... **much faster** than RNN.

In [31]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn.fit(X_train_pad, y_train, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )


res = model_cnn.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Epoch 1/20
55/55 [==============================] - 1s 6ms/step - loss: 0.6918 - accuracy: 0.5257 - val_loss: 0.6862 - val_accuracy: 0.5467
Epoch 2/20
55/55 [==============================] - 0s 5ms/step - loss: 0.5372 - accuracy: 0.8394 - val_loss: 0.5178 - val_accuracy: 0.7627
Epoch 3/20
55/55 [==============================] - 0s 5ms/step - loss: 0.2066 - accuracy: 0.9474 - val_loss: 0.4755 - val_accuracy: 0.7680
Epoch 4/20
55/55 [==============================] - 0s 5ms/step - loss: 0.0500 - accuracy: 0.9914 - val_loss: 0.5088 - val_accuracy: 0.7867
Epoch 5/20
55/55 [==============================] - 0s 5ms/step - loss: 0.0126 - accuracy: 0.9971 - val_loss: 0.4774 - val_accuracy: 0.8147
Epoch 6/20
55/55 [==============================] - 0s 5ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.5021 - val_accuracy: 0.8053
Epoch 7/20
55/55 [==============================] - 0s 5ms/step - loss: 3.0092e-04 - accuracy: 1.0000 - val_loss: 0.5803 - val_accuracy: 0.8133
Epoch 8/20
55/55

# Part 2 : Learn a Word2Vec representation, and then feed it into a NN with a `Conv1D`🔥 



In the first part of the exercise, you were asked to jointly learn the embedding representation and the CNN convolution within the same network, which was the CNN counterpart of the left part of this Figure: 

<img src="embedding_or_RNN.png" width="700px" />

Now, let's try to replace the RNN with a CNN for an architecture, as shown on the right side.



❓ **Question** ❓ Learn a Word2Vec model or load a trained one directly from GENSIM (transfer learning). Then, prepare your data as in the previous exercise. This question is quite long but it prepares you for real=world challenges. Don't worry, you have already done all the building bricks in the previous exercises!

In [52]:
from gensim.models import Word2Vec
import numpy as np

In [53]:
# YOUR CODE HERE
word2vec = Word2Vec(sentences=X_train,vector_size=150,min_count=4,window=7)

In [55]:
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)
X_train_pad = pad_sequences(X_train_embed, dtype='float32',maxlen=150)
X_test_pad = pad_sequences(X_test_embed, dtype='float32',maxlen=150)

❓ **Question** ❓ Now, build a model that has a `Conv1D` layer, a `Flatten` layer, a `Dense` layer, and an `output` layer. Compile and fit it on the train data. You can then evaluate it on the test set.

In [60]:
# YOUR CODE HERE
model = Sequential()

model.add(layers.Conv1D(64,kernel_size=4,activation='tanh'))
model.add(layers.Conv1D(128,kernel_size=4,activation='tanh'))
model.add(layers.Conv1D(256,kernel_size=4,activation='tanh'))
model.add(layers.Flatten())
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

❓ **Question** ❓ You might be frustrated by the accuracy you got, this happens to us all at some point. Once you have tested your first iteration, you need to improve your models: by making them more complex, changing the parameters, stacking additional layers, and so on...

Only practice and experimentation will get you there. So you can go back to your previous models, change them and try to get better results ;)

In [61]:
# YOUR CODE HERE
history = model.fit(X_train_pad,y_train,epochs=100,validation_split=0.33,batch_size=16,callbacks=[es])

Epoch 1/100
105/105 [==============================] - 3s 26ms/step - loss: 1.5075 - accuracy: 0.5197 - val_loss: 0.7186 - val_accuracy: 0.4709
Epoch 2/100
105/105 [==============================] - 3s 26ms/step - loss: 1.3650 - accuracy: 0.5550 - val_loss: 0.6979 - val_accuracy: 0.5630
Epoch 3/100
105/105 [==============================] - 3s 26ms/step - loss: 0.7050 - accuracy: 0.6099 - val_loss: 0.8286 - val_accuracy: 0.5242
Epoch 4/100
105/105 [==============================] - 3s 26ms/step - loss: 0.5899 - accuracy: 0.6941 - val_loss: 0.7639 - val_accuracy: 0.5484
Epoch 5/100
105/105 [==============================] - 3s 26ms/step - loss: 0.4504 - accuracy: 0.7879 - val_loss: 1.2373 - val_accuracy: 0.5206
Epoch 6/100
105/105 [==============================] - 3s 26ms/step - loss: 0.3118 - accuracy: 0.8590 - val_loss: 1.0373 - val_accuracy: 0.5400
Epoch 7/100
105/105 [==============================] - 3s 26ms/step - loss: 0.1966 - accuracy: 0.9229 - val_loss: 1.5938 - val_accuracy:

In [62]:
res = model.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 55.120%


🏁 Congratulations! Natural Language Processing is a complex topic and usually only a few students reach this challenge. You can be proud of yourself!

🐣 If you are reading this notebook after the bootcamp, it is also totally normal as there are plenty of concepts to learn and digest!

💾 Don't forget to `git add / commit / push`!